# 回顾

In [3]:
deployment="gpt4" #在azure中的deployment name
model="gpt-4"

In [3]:
import os
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("OPENAI_API_BASE")
    )

In [10]:
def translate(text):
    messages = []
    messages.append( {"role": "system", 
                      "content": "You are a translator. Please, translate the user's request to English."})
    messages.append( {"role": "user", "content": text})
    response = client.chat.completions.create(
        model=deployment,
        messages=messages,
        temperature=0.5,
        max_tokens = 100
    )
    return response.choices[0].message.content

In [11]:
import openai
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4")

print(encoding)

chinese = """在未来还没有到来的时候，总要有人把它创造出来，那个人应该是我们。"""
english = translate(chinese)

num_of_tokens_in_chinese = len(encoding.encode(chinese))
num_of_tokens_in_english = len(encoding.encode(english))

print(f"chinese:{chinese} ; {num_of_tokens_in_chinese} tokens\n")
print(f"english:{english} ; {num_of_tokens_in_english} tokens\n")

<Encoding 'cl100k_base'>
chinese:在未来还没有到来的时候，总要有人把它创造出来，那个人应该是我们。 ; 35 tokens

english:Before the future arrives, there always has to be someone to create it, and that person should be us. ; 22 tokens



# 常见应用场景

## 意图识别

In [12]:
import os
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("OPENAI_API_BASE")
    )

In [13]:
import openai
response = client.chat.completions.create(
    # 如果直接访问OpenAI GPT服务的同学，这里不要使用engine这个参数，要使用model，如： model=“gpt-4”
    model=deployment,
    temperature = 0,
    messages=[
        {"role": "system", "content": """
          Recognize the intent from the user's input 
         """},
        #{"role": "user", "content": "订明天早5点北京到上海的飞机"}
        {"role": "user", "content": "提醒我明早8点有会议"}
    ]
  )
print(response.choices[0].message.content)

用户的意图是设置一个提醒，以便在明天早上8点时提醒他们有一个会议。


## 生成SQL

In [15]:
import openai, os
from langchain.llms import OpenAI

system_prompt =  """  You are a software engineer, you can anwser the user request based on the given tables:
                  table “students“ with the columns [id, name, course_id, score] 
                  table "courses" with the columns [id, name] 
                  """

prompt = system_prompt

response = client.chat.completions.create(
    model=deployment,
    temperature = 0,
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": "计算所有学生英语课程的平均成绩"},
    ],
    max_tokens = 500
  )
print(response.choices[0].message.content)

为了计算所有学生英语课程的平均成绩，我们需要先找到英语课程对应的`course_id`，然后使用这个`course_id`去`students`表中筛选出所有选修了英语课程的学生，最后计算这些学生的成绩平均值。

以下是一个可能的SQL查询语句，用于计算英语课程的平均成绩：

```sql
SELECT AVG(s.score) AS average_score
FROM students s
JOIN courses c ON s.course_id = c.id
WHERE c.name = 'English';
```

这个查询语句做了以下几件事情：

1. 使用`JOIN`语句将`students`表和`courses`表连接起来，基于`course_id`和`id`字段的匹配。
2. 在`WHERE`子句中指定了课程名称为'English'，以筛选出所有英语课程的记录。
3. 使用`AVG`函数计算所有筛选出来的学生的成绩平均值，并将其命名为`average_score`。

执行这个查询后，你将得到所有学生英语课程的平均成绩。


## 生成代码友好的提示词

### 规范输出的格式

In [16]:
import openai
response = client.chat.completions.create(
    model=deployment,
    temperature = 0,
    messages=[
        {"role": "system", "content": """
          Recognize the intent from the user's input and format output as JSON string. 
        The output JSON string includes: "intention", "paramters" """},
        {"role": "user", "content": "提醒我明早8点有会议"}
    ]
  )
print(response.choices[0].message.content)

```json
{
  "intention": "set_reminder",
  "parameters": {
    "time": "8:00",
    "date": "tomorrow",
    "event": "会议"
  }
}
```


In [17]:
import openai, os
from langchain.llms import OpenAI

system_prompt =  """  You are a software engineer, you can write a SQL string as the anwser according to the user request 
               The user's requirement is based on the given tables:
                  table “students“ with the columns [id, name, course_id, score];
                  table "courses" with the columns [id, name]."""

prompt = system_prompt

response = client.chat.completions.create(
    model=deployment,
    temperature = 0,
    messages=[
        {"role": "system", "content": prompt},
        #{"role": "user", "content": "列出英语成绩大于80分的学生"},
        {"role": "user", "content": "列出英语课程成绩大于80分的学生, 返回结果只包括SQL"},
        #{"role": "user", "content": "列出年龄大于13的学生"}
    ],
    max_tokens = 500
  )
print(response.choices[0].message.content)

Sure, to list the students who scored more than 80 in the English course, you would need to join the `students` table with the `courses` table where the `course_id` matches the `id` of the English course, and then filter the results based on the score. Here is the SQL string that would accomplish this:

```sql
SELECT s.id, s.name, s.course_id, s.score
FROM students s
JOIN courses c ON s.course_id = c.id
WHERE c.name = 'English' AND s.score > 80;
```


### 文本规范异常输出的格式 

In [22]:
import openai, os
from langchain.llms import OpenAI

system_prompt =  """  You are a software engineer, you can write a SQL string as the anwser according to the user request 
               The user's requirement is based on the given tables:
                  table “students“ with the columns [id, name, course_id, score];
                  table "courses" with the columns [id, name]."""

system_prompt_with_negative =  """  
You are a software engineer, you can write a SQL string as the anwser according to the user request.
Also, when you cannot create the SQL query for the user's request based on the given tables, please, only return "invalid request"
               The user's requirement is based on the given tables:
                  table “students“ with the columns [id, name, course_id, score];
                  table "courses" with the columns [id, name]."""

#prompt = system_prompt
prompt = system_prompt_with_negative

response = openai.ChatCompletion.create(
    engine=deployment,
    model=model,
    temperature = 0,
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": "列出年龄大于13的学生"}
    ],
    max_tokens = 500
  )
print(response.choices[0].message.content)

-1
